## Generates the poule sheets for Judo+ tournaments  
#### Work by [Gonçalo Sousa](https://github.com/Sousa1909) for [AJDS](https://github.com/AJD-Santarem)  

#### Script 2
---

In [24]:
import yaml
import os
import pandas as pd
from openpyxl import load_workbook

# Tournament folder
tournament_folder = "REPLACE_ME"

# Define output folder
output_folder = "Output/"

# File paths for the Excel templates
best_of_3 = "Templates/BO_3.xlsx"
poule_of_5 = "Templates/Poule_5.xlsx"

In [ ]:
# Load the appropriate template based on group size
def load_template(group_size, template_file_5, template_file_2):
    if group_size == 2:
        return True, load_workbook(template_file_2)
    else:
        return False, load_workbook(template_file_5)

# Populate the table with the group information
def populate_group_table(group_data, template_file_5, template_file_2, output_file, group_number, age_tier):
    # Determine group size based on the number of athletes in the group
    bo3, workbook = load_template(len(group_data), template_file_5, template_file_2)
    sheet = workbook.active

    # Fill in athlete data based on the loaded template
    if bo3:
        for i, athlete in enumerate(group_data, start=8):
            sheet[f'C{i}'] = athlete['Name']
            sheet[f'D{i}'] = athlete['Club']
        sheet[f'F{13}'] = group_number
        sheet[f'J{4}'] = age_tier
    else:
        for i, athlete in enumerate(group_data, start=9):
            sheet[f'B{i}'] = athlete['Name']
            sheet[f'C{i}'] = athlete['Club']
        sheet[f'L{17}'] = group_number
        sheet[f'M{4}'] = age_tier

    # Save the updated file to the designated folder
    output_filename = os.path.join(output_folder + tournament_folder, output_file.format(age_tier, group_number))
    workbook.save(output_filename)
    print(f"Group {group_number} saved to {output_filename}")

# Function to read groups from the YAML file
def read_groups_from_yaml(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = yaml.safe_load(f)

    # Extract groups by age tier
    groups_by_age_tier = []
    for age_tier, groups in data.items():
        age_tier_groups = []
        for group_info in groups:
            # Extract group key (e.g., 'Group 1') and its athletes
            for group_key, athletes in group_info.items():
                age_tier_groups.append(pd.DataFrame(athletes))
        groups_by_age_tier.append((age_tier, age_tier_groups))

    return groups_by_age_tier

# Function to process each group and generate tables from YAML data
def create_tables_from_yaml(yaml_file_path, template_file_5, template_file_2, output_file_pattern):
    groups_by_age_tier = read_groups_from_yaml(yaml_file_path)
    for age_tier, groups in groups_by_age_tier:
        group_number = 1
        for group in groups:
            # Populate and save each group table
            populate_group_table(group.to_dict(orient='records'), template_file_5, template_file_2, output_file_pattern, group_number, age_tier)
            group_number += 1

# Generate tables for both masculine and feminine groups
create_tables_from_yaml(f'{output_folder + tournament_folder}/.grouped_athletes_mas.yaml', poule_of_5, best_of_3, 'Group_M_{}_{}.xlsx')
create_tables_from_yaml(f'{output_folder + tournament_folder}/.grouped_athletes_fem.yaml', poule_of_5, best_of_3, 'Group_F_{}_{}.xlsx')
